# Data Aggregation and Group Operations

In [4]:
import pandas as pd
import numpy as np

## GroupBy mechanics

We are going to review the Group By (simmilar to the one in SQL)

In [7]:
df = pd.DataFrame({'key1' : list('aabba'),
                   'key2' : ['one','two','one','two','one'],
                   'balance' : np.random.randn(5) * 10,
                   'income' : np.random.randn(5) + 2
    
})

df

,balance,income,key1,key2
0,6.657970,1.399476,a,one
1,0.621236,4.367726,a,two
2,-8.940374,1.175115,b,one
3,5.069804,2.965204,b,two
4,6.427537,0.558563,a,one


In [8]:
df.mean()

balance    1.967235
income     2.093217
dtype: float64

In [10]:
df.groupby('key1')

It gives a lazy collection. It is waiting a method to display something

In [11]:
df.groupby('key1').mean()

,balance,income
key1,,
a,4.568914,2.108589
b,-1.935285,2.070160


In [13]:
mean_key = df.groupby('key1')['balance'].mean()
mean_key

key1
a    4.568914
b   -1.935285
Name: balance, dtype: float64

In [14]:
mean_key['a']

4.5689141007453786

In [16]:
df.groupby(['key1','key2']).agg(['mean','count'])

balance          income      
               mean count      mean count
key1 key2                                
a    one   6.542753     2  0.979020     2
     two   0.621236     1  4.367726     1
b    one  -8.940374     1  1.175115     1
     two   5.069804     1  2.965204     1

In [19]:
help(mean_key.agg)

Help on method aggregate in module pandas.core.series:

aggregate(func, axis=0, *args, **kwargs) method of pandas.core.series.Series instance
    Aggregate using callable, string, dict, or list of string/callables
    
    .. versionadded:: 0.20.0
    
    Parameters
    ----------
    func : callable, string, dictionary, or list of string/callables
        Function to use for aggregating the data. If a function, must either
        work when passed a Series or when passed to Series.apply. For
        a DataFrame, can pass a dict, if the keys are DataFrame column names.
    
        Accepted Combinations are:
    
        - string function name
        - function
        - list of functions
        - dict of column names -> functions (or list of functions)
    
    Notes
    -----
    Numpy functions mean/median/prod/sum/std/var are special cased so the
    default behavior is applying the function along axis=0
    (e.g., np.mean(arr_2d, axis=0)) as opposed to
    mimicking the default

In [22]:
df.groupby('key1')['key2'].agg(lambda strseries: strseries.str.len().sum())

key1
a    9
b    6
Name: key2, dtype: int64

### Iterating over groups

In [25]:
for xxx in df.groupby('key1'):
    print (type(xxx))
    print (xxx)

<class 'tuple'>
('a',     balance    income key1 key2
0  6.657970  1.399476    a  one
1  0.621236  4.367726    a  two
4  6.427537  0.558563    a  one)
<class 'tuple'>
('b',     balance    income key1 key2
2 -8.940374  1.175115    b  one
3  5.069804  2.965204    b  two)


When iterating, it is taken the tuples of the key:values

In [26]:
for value, group in df.groupby('key1'):
    print (value)
    print (group)

a
    balance    income key1 key2
0  6.657970  1.399476    a  one
1  0.621236  4.367726    a  two
4  6.427537  0.558563    a  one
b
    balance    income key1 key2
2 -8.940374  1.175115    b  one
3  5.069804  2.965204    b  two


In [27]:
a = list(df.groupby('key1')) #we need to make the lazy objects to run
a

[('a',     balance    income key1 key2
  0  6.657970  1.399476    a  one
  1  0.621236  4.367726    a  two
  4  6.427537  0.558563    a  one), ('b',     balance    income key1 key2
  2 -8.940374  1.175115    b  one
  3  5.069804  2.965204    b  two)]

In [29]:
dict_groupby = dict(a)
dict_groupby

{'a':     balance    income key1 key2
 0  6.657970  1.399476    a  one
 1  0.621236  4.367726    a  two
 4  6.427537  0.558563    a  one, 'b':     balance    income key1 key2
 2 -8.940374  1.175115    b  one
 3  5.069804  2.965204    b  two}

## Data aggregation

In [31]:
import requests

url = 'https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv'
response = requests.get(url)

out_file = open('tips.csv','wb')
out_file.write(response.content)
out_file.close()

In [32]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Are the females and males giving different tips

In [52]:
tips['perc_tip'] = tips['tip'] / tips['total_bill']
values_for_z_stat = tips.groupby('sex').agg(['mean','std','count'])['perc_tip']
z_stat = (values_for_z_stat['mean']['Male']-values_for_z_stat['mean']['Female'])/(tips['perc_tip'].std()/np.sqrt(tips['perc_tip'].count()))
z_stat
values_for_z_stat 

,mean,std,count
sex,,,
Female,0.166491,0.053632,87
Male,0.157651,0.064778,157


In [56]:
stacked = df.groupby(['key1','key2']).mean()
stacked

balance    income
key1 key2                    
a    one   6.542753  0.979020
     two   0.621236  4.367726
b    one  -8.940374  1.175115
     two   5.069804  2.965204

In [60]:
stacked.unstack()

balance              income          
key2       one       two       one       two
key1                                        
a     6.542753  0.621236  0.979020  4.367726
b    -8.940374  5.069804  1.175115  2.965204

In [61]:
stacked.unstack('key1').unstack('balance')

         key1  key2
balance  a     one     6.542753
               two     0.621236
         b     one    -8.940374
               two     5.069804
income   a     one     0.979020
               two     4.367726
         b     one     1.175115
               two     2.965204
dtype: float64

In [63]:
tips.pivot(columns = 'sex')

total_bill           tip       smoker          day          time          \
sex     Female   Male Female  Male Female  Male Female  Male  Female    Male   
0        16.99    NaN   1.01   NaN     No  None    Sun  None  Dinner    None   
1          NaN  10.34    NaN  1.66   None    No   None   Sun    None  Dinner   
2          NaN  21.01    NaN  3.50   None    No   None   Sun    None  Dinner   
3          NaN  23.68    NaN  3.31   None    No   None   Sun    None  Dinner   
4        24.59    NaN   3.61   NaN     No  None    Sun  None  Dinner    None   
5          NaN  25.29    NaN  4.71   None    No   None   Sun    None  Dinner   
6          NaN   8.77    NaN  2.00   None    No   None   Sun    None  Dinner   
7          NaN  26.88    NaN  3.12   None    No   None   Sun    None  Dinner   
8          NaN  15.04    NaN  1.96   None    No   None   Sun    None  Dinner   
9          NaN  14.78    NaN  3.23   None    No   None   Sun    None  Dinner   
10         NaN  10.27    NaN  1.71   None    No   None   Sun    None  Dinner   
11       35.26    NaN   5.00   NaN     No  None    Sun  None  Dinner    None   
12         NaN  15.42    NaN  1.57   None    No   None   Sun    None  Dinner   
13         NaN  18.43    NaN  3.00   None    No   None   Sun    None  Dinner   
14       14.83    NaN   3.02   NaN     No  None    Sun  None  Dinner    None   
15         NaN  21.58    NaN  3.92   None    No   None   Sun    None  Dinner   
16       10.33    NaN   1.67   NaN     No  None    Sun  None  Dinner    None   
17         NaN  16.29    NaN  3.71   None    No   None   Sun    None  Dinner   
18       16.97    NaN   3.50   NaN     No  None    Sun  None  Dinner    None   
19         NaN  20.65    NaN  3.35   None    No   None   Sat    None  Dinner   
20         NaN  17.92    NaN  4.08   None    No   None   Sat    None  Dinner   
21       20.29    NaN   2.75   NaN     No  None    Sat  None  Dinner    None   
22       15.77    NaN   2.23   NaN     No  None    Sat  None  Dinner    None   
23         NaN  39.42    NaN  7.58   None    No   None   Sat    None  Dinner   
24         NaN  19.82    NaN  3.18   None    No   None   Sat    None  Dinner   
25         NaN  17.81    NaN  2.34   None    No   None   Sat    None  Dinner   
26         NaN  13.37    NaN  2.00   None    No   None   Sat    None  Dinner   
27         NaN  12.69    NaN  2.00   None    No   None   Sat    None  Dinner   
28         NaN  21.70    NaN  4.30   None    No   None   Sat    None  Dinner   
29       19.65    NaN   3.00   NaN     No  None    Sat  None  Dinner    None   
..         ...    ...    ...   ...    ...   ...    ...   ...     ...     ...   
214      28.17    NaN   6.50   NaN    Yes  None    Sat  None  Dinner    None   
215      12.90    NaN   1.10   NaN    Yes  None    Sat  None  Dinner    None   
216        NaN  28.15    NaN  3.00   None   Yes   None   Sat    None  Dinner   
217        NaN  11.59    NaN  1.50   None   Yes   None   Sat    None  Dinner   
218        NaN   7.74    NaN  1.44   None   Yes   None   Sat    None  Dinner   
219      30.14    NaN   3.09   NaN    Yes  None    Sat  None  Dinner    None   
220        NaN  12.16    NaN  2.20   None   Yes   None   Fri    None   Lunch   
221      13.42    NaN   3.48   NaN    Yes  None    Fri  None   Lunch    None   
222        NaN   8.58    NaN  1.92   None   Yes   None   Fri    None   Lunch   
223      15.98    NaN   3.00   NaN     No  None    Fri  None   Lunch    None   
224        NaN  13.42    NaN  1.58   None   Yes   None   Fri    None   Lunch   
225      16.27    NaN   2.50   NaN    Yes  None    Fri  None   Lunch    None   
226      10.09    NaN   2.00   NaN    Yes  None    Fri  None   Lunch    None   
227        NaN  20.45    NaN  3.00   None    No   None   Sat    None  Dinner   
228        NaN  13.28    NaN  2.72   None    No   None   Sat    None  Dinner   
229      22.12    NaN   2.88   NaN    Yes  None    Sat  None  Dinner    None   
230        NaN  24.01    NaN  2.00   None   Yes   None   Sat    None  Dinner   
231 

## Group-wise operations and transformations

### Apply: General split-apply-combine

#### Suppressing the group keys

### Quantile and bucket analysis

### Example: Filling missing values with group-specific values

## Pivot tables and Cross-tabulation